<h1>Redes Sociais: APS 2</h1>

<h3>Professor: Marcelo Hashimoto</h3>

<h3>Alunos: Bernardo Capoferri, Nicolas Queiroga e Raphael Lahiry<h3>

<h1>Carregando o Grafo</h1>

In [2]:
# Importando bibliotecas necessárias para a construção de grafos
import netpixi
import graph_tool.topology
from netpixi.integration.gt import *
from regression.integration.gt import *

/usr/local/lib/python3.10/site-packages/seaborn/rcmod.py:4: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


In [4]:
g = gt_load('graph_us_roads.net.gz');

<h1>Reduzindo o Grafo a uma Componente e posicionando os vértices baseado em force-directed graph drawing</h1>

In [3]:
comp, hist = graph_tool.topology.label_components(g, directed=False)

In [4]:
sum_dic = {}
for node in comp.a:
    if node not in sum_dic:
        sum_dic[node] = 1
    else:
        sum_dic[node] += 1

componente = sorted(sum_dic, key=sum_dic.get)[-2]

In [5]:
# Leitura e do arquivo e construção do grafo

graph_us_roads_component = Graph(directed=False) # grafo não-dirigido

with open('./road-usroads.mtx') as file:
    
    # Pula primeiras 14 linhas para chegar nas linhas com os dados do grafo
    for i in range(14):
        next(file)
        
    graph_data = file.readline().split()
    n_nodes = int(graph_data[1])
    n_edges = int(graph_data[2])
    
    
    # Adicionando vétices ao grafo
    for i in range(n_nodes):
        if comp.a[i] == componente:
            graph_us_roads_component.add_vertex(i+1)
    
    # Adicionando arestas ao grafo
    for edges in file:
        edge_data = edges.split()
        if comp.a[int(edge_data[0])-1] == componente and comp.a[int(edge_data[1])-1] == componente:
            graph_us_roads_component.add_edge(int(edge_data[0]), int(edge_data[1]))

print('Número de vértices do grafo:', graph_us_roads_component.num_vertices())
print('Número de arestas do grafo:', graph_us_roads_component.num_edges())

Número de vértices do grafo: 1923
Número de arestas do grafo: 2279


In [6]:
m = gt_draw.sfdp_layout(graph_us_roads_component)
gt_move(graph_us_roads_component, m)
gt_save(graph_us_roads_component, 'graph_us_roads_component_menor.net.gz')

In [6]:
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

In [9]:
print('Número de vértices do grafo:', g.num_vertices())
print('Número de arestas do grafo:', g.num_edges())

Número de vértices do grafo: 126146
Número de arestas do grafo: 161950


<h1>Questões</h1>

<h2>a. O que os vértices da rede representam?</h2>

<h1>Carregando o Grafo</h1>

Os vértices da rede representam ruas dos Estados Unidos.

<h2>b. O que as arestas da rede representam?</h2>

As arestas da rede indicam conexões entre ruas dos Estados Unidos.

<h2>c. O que os grupos formados pela tarefa 4 representam?</h2>

Os grupos formados pela tarefa 4 representam uma grande densidade de ruas que se conectam, muito possivelmente configurando um região urbana.

<h1>Métricas</h1>

<h2>Centralidade Degree</h2>

<h3>a. Calculando a Métrica</h3>

In [10]:
dc = gt_total_degree(g)
dc_menor = gt_total_degree(g_menor)

In [11]:
g.add_vp('degree', dc)
g_menor.add_vp('degree', dc_menor)

In [12]:
gt_save(g_menor, 'graph_degree_component_menor.net.gz')

In [19]:
data = gt_data(g).sort_values(by="degree", ascending=True)
data

,label,degree
id,,
532,532,0.000008
42681,42681,0.000008
42750,42750,0.000008
42776,42776,0.000008
42802,42802,0.000008
...,...,...
111891,111891,0.000048
102661,102661,0.000048
42386,42386,0.000048


<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [34]:
r = netpixi.render('graph_degree_component_menor.net.gz', infinite=True)

In [36]:
r.vertex_scale('degree', 10, 50)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

A métrica degree nos fala quantas arestas estão conectadas à um vértice. No nosso caso, o degree de um vértice representaria a quantidade de ruas ligadas a ele.

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

Sim. Representando as ruas com maiores conexões, os maiores vértices parecem estar localizados ou em clusters de vértices, ou seja, agrupamentos mais densos de ruas, ou em regiões de conexão entre outros clusters.

<h2>Centralidade Betweenness</h2>

<h3>a. Calculando a Métrica</h3>

In [ ]:
from graph_tool import centrality
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

bc, _ = centrality.betweenness(g)
bc_menor, _ = centrality.betweenness(g_menor)

In [23]:
g.add_vp('betweenness', bc)
g_menor.add_vp('betweenness', bc_menor)

In [24]:
gt_save(g_menor, 'graph_betweeness_component_menor.net.gz')

In [3]:
g = gt_load('graph_betweeness_component.net.gz')

In [8]:
data = gt_data(g).sort_values(by="betweenness", ascending=False)
data

,label,betweenness
id,,
77580,77580,0.213117
78746,78746,0.211596
78261,78261,0.206446
77901,77901,0.206056
78026,78026,0.205337
...,...,...
79419,79419,0.000000
79456,79456,0.000000
79763,79763,0.000000


<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [37]:
r = netpixi.render('graph_betweeness_component_menor.net.gz', infinite=True)

In [38]:
r.vertex_scale('betweenness', 10, 50)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

A métrica representa a soma das proporções de geodésicas que passam pelo nosso vértice. Uma geodésica de um vértice X para um vértice Y é o caminho de menor comprimento possível. No caso da nossa rede, um vértice com um betweenness alto representa uma rua que tem uma proximidade grande com as ruas as quais se conecta.

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

<h2>Centralidade Closeness</h2>

<h3>a. Calculando a Métrica</h3>

In [29]:
from graph_tool import centrality
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

cc = centrality.closeness(g)
cc_menor = centrality.closeness(g_menor)

In [27]:
hcc = centrality.closeness(g, harmonic=True)

0.005147415032613062
0.005147415032613062
0.004387608891006128
0.005105780831523639
0.004387608891006128
0.005893756149486838
0.004427257462269793
0.006083235287533051
0.006032120099728592
0.006212208679685855
0.006363696747807139
0.0063698169196460906
0.0059019044923445145
0.005855839824308918
0.005759487277645148
0.0064288522045490215
0.005218326171202024
0.0059740444865633196
0.006278436135102076
0.006337965865348435
0.005893756149486838
0.00629742781272384
0.006254355031043508
0.0062117402145394586
0.00628402935424991
0.006005448450114075
0.006214925554155263
0.006168880685187336
0.006243584637085749
0.006203379854799434
0.006283800750456623
0.006404666362385797
0.00635432531936453
0.006073999346311374
0.006186829745584442
0.006256543294093661
0.006433273591830317
0.0063170795513024965
0.006026639901681334
0.006358630497003724
0.006172390865541953
0.006296590621433038
0.006219360234740808
0.006256140061638624
0.005849394327380006
0.005946848789570165
0.00446767229723323
0.004432410

In [39]:
g.add_vp('closeness', cc)
g_menor.add_vp('closeness', cc_menor)

In [40]:
gt_save(g_menor, 'graph_closeness_component_menor.net.gz')

<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [41]:
r = netpixi.render('graph_closeness_component_menor.net.gz', infinite=True)

In [42]:
r.vertex_scale('closeness', 10, 50)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

<h2>Clustering Coefficient</h2>

<h3>a. Calculando a Métrica</h3>

In [43]:
from graph_tool import clustering
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

lc = clustering.local_clustering(g)
lc_menor = clustering.local_clustering(g_menor)

In [44]:
g.add_vp('clustering', lc)
g_menor.add_vp('clustering', lc_menor)

In [45]:
gt_save(g_menor, 'graph_clustering_component_menor.net.gz')

<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [46]:
r = netpixi.render('graph_clustering_component_menor.net.gz', infinite=True)

In [47]:
r.vertex_scale('clustering', 10, 30)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

<h2>Lacuas Estruturais</h2>

<h3>a. Calculando a Métrica</h3>

In [48]:
g = gt_load("graph_us_roads_component.net.gz")
g_menor = gt_load("graph_us_roads_component_menor.net.gz")

es = gt_effective_size(g)
es_menor = gt_effective_size(g_menor)

In [49]:
g.add_vp('effsize', es)
g_menor.add_vp('effsize', es_menor)

In [50]:
gt_save(g_menor, 'graph_effective_size_component_menor.net.gz')

<h3>b. Chamando o método vertex_scale da netpixi para usar a métrica como referência para os tamanhos dos vértices</h3>

In [53]:
r = netpixi.render('graph_effective_size_component_menor.net.gz', infinite=True)

In [54]:
r.vertex_scale('effsize', 10, 30)

<h3>c. Salvando uma imagem que mostre com clareza essas diferenças de tamanho</h3>

<h3>d.1. A métrica representa algum conceito? Se sim, qual?</h3>

<h3>d.2. Os maiores vértices (considerando a tarefa 6b) parecem estar em posições interessantes? Quais?</h3>

Os grupos formados pela tarefa 4 representam uma grande densidade de ruas, configurando uma região urbana. Possivelmente uma cidade, mas não tem como ter certeza.

<h3>Referências e reconhecimento<h3>

@inproceedings{nr,
     title={The Network Data Repository with Interactive Graph Analytics and Visualization},
     author={Ryan A. Rossi and Nesreen K. Ahmed},
     booktitle={AAAI},
     url={https://networkrepository.com},
     year={2015}
}

Link: https://networkrepository.com/road-usroads.php